In [17]:
import os
path = 'C:/pytest/word2vec/'
os.chdir(path)

In [2]:
import pandas as pd
import numpy as np

In [12]:
from gensim.models import Word2Vec

In [5]:
with open('wiki_test.txt','r',encoding = 'utf-8') as f:
    data = f.read()

In [6]:
embedding_dim = 200

In [8]:
import rhinoMorph
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\광주인공지능사관학교\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [9]:
sent_data = sent_tokenize(data)
rn = rhinoMorph.startRhino()

filepath:  C:\Anaconda3\lib\site-packages
classpath:  C:\Anaconda3\lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!


In [10]:
print(type(sent_data), len(sent_data), sent_data[:20], sep = '\n')

<class 'list'>
11977
['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">\n지미 카터\n\n제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.', '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.', '조지아 공과대학교를 졸업하였다.', '그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다.', '1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.', '그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', '1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다.', '대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다.', '조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.', '1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.', '카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.', '카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.', '그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다.', '1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다.', '또한 소련과 제2차 전략 무기 제한 협상에 조인했다.', '카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.', '<!DOCTYPE tei.2  SYSTEM "c:굎gml괺td굏ei2.dt

In [13]:
# os.mkdir('./word2vec')

# Word2Vector

In [14]:
from tqdm import tqdm
with open('./word2vec/wiki_test_morphed.txt','w', encoding = 'utf-8') as f:
    for data_each in tqdm(sent_data):
        morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each, pos = ['NNP','NNG','VV','VA','XR','IC','MM','MAG','MAJ'],
                                                     xrVv = True, eomi = True)
        joined_data_each = ' '.join(morphed_data_each)
        if joined_data_each:
            f.write(joined_data_each + '\n')

100%|███████████████████████████████████████████████████████████████████████████| 11977/11977 [00:27<00:00, 432.65it/s]


In [15]:
def read_data(filename, encoding = 'utf-8'):
    with open(filename, 'r', encoding = encoding) as f:
        data = [line.split(' ') for line in f.read().splitlines()]
    return data

In [16]:
data = read_data('./word2vec/wiki_test_morphed.txt','utf-8')
print(len(data), type(data), data[:3], sep = '\n')

11972
<class 'list'>
[['URL', '미', '카터', '미', '카터', '제임스', '얼다', '미', '카터', '주니어', '민주당', '출신', '미국', '대통령'], ['미', '카터', '조지아', '섬터', '카운티', '마을', '태어나다'], ['조지아', '공과대학', '교', '졸업하다']]


In [18]:
model = Word2Vec(sentences= data, vector_size= embedding_dim, window= 10, min_count= 5, workers= 4 , sg= 1)
model.save('embedding200_window10_mincnt5_skipgram.model')
# embedding200_window10_mincnt5_skipgram.model.wv.vectors.npy
# 단어와 임베딩 값을 가지고 있는 파일
# embedding200_window10_mincnt5_skipgram.model.syn1neg.npy
# 임베딩 값을 계산하기 위해 사용된 신경망 네트워크와 가중치 보유

In [19]:
words = list(model.wv.index_to_key)
with open('embedding200_window10_mincnt5_skipgram.txt','w') as f:
    for word in tqdm(words):
        data = model.wv[word].tolist() # 200개의 가중치를 리스트로
        data.insert(0, word) # 데이터의 0번 위치에 해당 단어를 끼워 넣음
                             # 알아보기 쉽게 함
        for item in data: # 단어 이름부터 시작하여 각 벡터 값을 저장
            f.write('%s ' % item)
        f.write('\n')

100%|██████████████████████████████████████████████████████████████████████████| 10643/10643 [00:02<00:00, 4046.61it/s]


In [22]:
model = Word2Vec.load('embedding200_window10_mincnt5_skipgram.model')

print('--- 유사단어 출력 ---')
similarWords = model.wv.most_similar(positive= ['행복','웃음','밝다','기쁨'], topn=10)
print(similarWords) # 확률값, 정도값

--- 유사단어 출력 ---
[('행복하다', 0.9828894138336182), ('맴돌다', 0.9787048697471619), ('고뇌', 0.9784281849861145), ('감동', 0.9782537817955017), ('따뜻하다', 0.9779002666473389), ('소망', 0.97676682472229), ('두텁다', 0.9756407737731934), ('선비', 0.9752475023269653), ('애꾸눈', 0.9741258025169373), ('설레다', 0.9733781814575195)]


In [23]:
word = []
for similarWord in similarWords:
    word.append(similarWord[0])
print(word)

['행복하다', '맴돌다', '고뇌', '감동', '따뜻하다', '소망', '두텁다', '선비', '애꾸눈', '설레다']


In [24]:
print('--- 두 단어의 유사도 계산 ---\n한국과 일본 : {}\n한국과 미국 : {}\n한국과 중국 : {}'.format(model.wv.similarity('한국','일본'),model.wv.similarity('한국','미국'),model.wv.similarity('한국','중국')))

--- 두 단어의 유사도 계산 ---
한국과 일본 : 0.5648971796035767
한국과 미국 : 0.6623880863189697
한국과 중국 : 0.705439031124115


In [25]:
model = Word2Vec.load('embedding_window10_mincnt5_skipgram_big.model')
print('--- 유사단어 출력 ---')
similarWords = model.wv.most_similar(positive= ['행복','웃음','밝다','기쁨'], topn=10)
print(similarWords) # 확률값, 정도값
word = []
for similarWord in similarWords:
    word.append(similarWord[0])
print(word)

--- 유사단어 출력 ---
[('행복하다', 0.7607556581497192), ('기쁘다', 0.7480962872505188), ('즐겁다', 0.7307630181312561), ('흐뭇하다', 0.7170217037200928), ('즐거움', 0.7020072340965271), ('자약하다', 0.6902682781219482), ('아름답다', 0.676089346408844), ('법열감', 0.6757972836494446), ('유쾌하다', 0.6678250432014465), ('해맑다', 0.6666801571846008)]
['행복하다', '기쁘다', '즐겁다', '흐뭇하다', '즐거움', '자약하다', '아름답다', '법열감', '유쾌하다', '해맑다']


In [26]:
print('--- 두 단어의 유사도 계산 ---\n한국과 일본 : {}\n한국과 미국 : {}\n한국과 중국 : {}'.format(model.wv.similarity('한국','일본'),model.wv.similarity('한국','미국'),model.wv.similarity('한국','중국')))

--- 두 단어의 유사도 계산 ---
한국과 일본 : 0.7467085123062134
한국과 미국 : 0.6853418350219727
한국과 중국 : 0.5778185129165649


# FastText

In [28]:
# os.mkdir('C:/pytest/fasttext/')

In [29]:
os.chdir('C:/pytest/fasttext/')

In [34]:
data = read_data('C:/pytest/word2vec/wiki_test_morphed.txt','utf-8')
print(len(data), type(data), data[:3], sep = '\n')

11972
<class 'list'>
[['URL', '미', '카터', '미', '카터', '제임스', '얼다', '미', '카터', '주니어', '민주당', '출신', '미국', '대통령'], ['미', '카터', '조지아', '섬터', '카운티', '마을', '태어나다'], ['조지아', '공과대학', '교', '졸업하다']]


In [36]:
from gensim.models import FastText
model = FastText(sentences= data, vector_size= embedding_dim, window= 10, min_count= 2, workers= 4)
model.save('embedding200_window10_mincnt2_fasttext.model')

In [37]:
model = Word2Vec.load('embedding200_window10_mincnt2_fasttext.model')
print('--- 유사단어 출력 ---')
similarWords = model.wv.most_similar(positive= ['행복','웃음','밝다','기쁨'], topn=10)
print(similarWords) # 확률값, 정도값
word = []
for similarWord in similarWords:
    word.append(similarWord[0])
print(word)

--- 유사단어 출력 ---
[('젊은', 0.9997346997261047), ('키우다', 0.9997287392616272), ('채우다', 0.9997075796127319), ('씌우다', 0.9996822476387024), ('거세다', 0.9996662139892578), ('길이', 0.9996564388275146), ('배우다', 0.999651312828064), ('북돋우다', 0.9996458292007446), ('몰려들다', 0.9996426105499268), ('치켜세우다', 0.9996381998062134)]
['젊은', '키우다', '채우다', '씌우다', '거세다', '길이', '배우다', '북돋우다', '몰려들다', '치켜세우다']


In [38]:
print('--- 두 단어의 유사도 계산 ---\n한국과 일본 : {}\n한국과 미국 : {}\n한국과 중국 : {}'.format(model.wv.similarity('한국','일본'),model.wv.similarity('한국','미국'),model.wv.similarity('한국','중국')))

--- 두 단어의 유사도 계산 ---
한국과 일본 : 0.9854395389556885
한국과 미국 : 0.9394056797027588
한국과 중국 : 0.9936444163322449
